In [1]:
import numpy as np
from sklearn import datasets
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import cross_val_predict, cross_validate, cross_val_score
from sklearn.model_selection import KFold
import sklearn.metrics as metrics
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from pprint import pprint
from sklearn.svm import SVC

In [2]:
# Carga de datos.
datos = datasets.load_iris()
print(np.shape(datos.data))

(150, 4)


In [3]:
# Extraer la matriz de datos "X" y la variable target "y"
X = datos.data
y = datos.target

In [4]:
# Partición EXTERNA. Test: hold-out split 80-20%.
X_train, X_test, y_train, y_test = train_test_split(datos.data, datos.target, test_size=0.2, random_state=42)
print('Train dimensions: ', np.shape(X_train))
print('Test dimensions:  ', np.shape(X_test))

Train dimensions:  (120, 4)
Test dimensions:   (30, 4)


In [5]:
# Estandarización de los datos de entrenamiento y test
standardizer = preprocessing.StandardScaler()
X_train = standardizer.fit_transform(X_train)
X_test = standardizer.transform(X_test)

In [6]:
# Definimos el algoritmo
alg = SVC(C=0.1, gamma='auto', kernel='rbf')

In [7]:
# Cross-validation interno
# Utilizar comando "cross_val_predict" para extraer directamente las predicciones de las 5 bolsas de validación
y_pred = cross_val_predict(alg, X_train, y_train, cv= KFold(n_splits=5, shuffle=True, random_state=42))
pprint(y_pred)

array([0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 2, 1, 1, 0, 0, 1, 1, 2, 1, 2, 1, 2,
       1, 0, 2, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 2, 0, 1, 2, 0, 1, 1,
       1, 1, 2, 1, 0, 1, 2, 0, 0, 1, 2, 0, 2, 0, 0, 1, 1, 2, 1, 1, 2, 1,
       0, 0, 1, 2, 0, 0, 0, 1, 2, 0, 2, 2, 0, 1, 1, 1, 2, 2, 0, 2, 1, 2,
       1, 1, 1, 0, 1, 1, 0, 1, 2, 2, 0, 1, 2, 2, 0, 2, 0, 2, 2, 2, 1, 2,
       1, 1, 1, 1, 0, 1, 1, 0, 1, 2])


In [8]:
# Presentación de los resultados de la evaluación.
print("Exactitud: %.3f" % (metrics.accuracy_score(y_train, y_pred))) # accuracy
print("Precisión: %.3f" % (metrics.precision_score(y_train, y_pred, average="macro"))) # precision
print("Sensibilidad: %.3f" % (metrics.recall_score(y_train, y_pred, average="macro"))) # sensibilidad
print("F1-score: %.3f" % (metrics.f1_score(y_train, y_pred, average="macro"))) # F-score

Exactitud: 0.883
Precisión: 0.893
Sensibilidad: 0.882
F1-score: 0.881


In [9]:
# Extraemos la matriz de confusión
print("Matriz de confusión:\n", metrics.confusion_matrix(y_train, y_pred))

Matriz de confusión:
 [[40  0  0]
 [ 0 38  3]
 [ 0 11 28]]


In [10]:
print("Tabla de métricas:\n", metrics.classification_report(y_train, y_pred))

Tabla de métricas:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        40
           1       0.78      0.93      0.84        41
           2       0.90      0.72      0.80        39

    accuracy                           0.88       120
   macro avg       0.89      0.88      0.88       120
weighted avg       0.89      0.88      0.88       120



In [11]:
# Una vez hemos optimizado los hiperparámetros con el conjunto de validación, utilizamos todos los datos de 
# "train" y "validation" para entrenar un modelo definitivo con esos hiperparámetros ajustados
model = SVC(C=0.1, gamma='auto', kernel='rbf') # Definimos el modelo con los hiperparámetros óptimos
model = model.fit(X_train, y_train) # Entrenamos el modelo con todos los datos de "train" + "val"

# Evaluamos el test con el método "score()"
test_results = model.score(X_test, y_test)
print('Exactitud en test: ', np.round(test_results*100,4), '%')

Exactitud en test:  96.6667 %


In [18]:
# obtenemos las predicciones del test para extraer el resto de métricas de interés
y_pred_test = model.predict(X_test) # Extraer predicciones
print("Tabla de métricas:\n", metrics.classification_report(y_test, y_pred_test)) # Aplicar el método de classification_report()
print("Matriz de confusión:\n", metrics.confusion_matrix(y_test, y_pred_test)) # Extraer la matriz de confusión

Tabla de métricas:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       0.90      1.00      0.95         9
           2       1.00      0.91      0.95        11

    accuracy                           0.97        30
   macro avg       0.97      0.97      0.97        30
weighted avg       0.97      0.97      0.97        30

Matriz de confusión:
 [[10  0  0]
 [ 0  9  0]
 [ 0  1 10]]
